In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models import KeyedVectors

In [2]:
from src.data_processing.process_labels import *
from src.data_processing.process_reviews import *

## Load in saved LSTM torch model

In [31]:
from src.models.model_zoo import *
from src.models.model_train import *

# instatiate model
input_size = 300
hidden_size = 10
num_layers = 2
dropout_prob = 0.3
output_size = 4
lstm_model = LSTMmodel(input_size=input_size, hidden_size=hidden_size, 
                       output_size=output_size, num_layers=num_layers, dropout_prob=dropout_prob)

In [32]:
count_parameters(lstm_model)

27604

In [16]:
# load saved parameters
lstm_model.load_state_dict(torch.load('jobs/out_h300_l2_dp03.0.pt'))
lstm_model.eval()

LSTMmodel(
  (LSTM): LSTM(300, 300, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=600, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

## Sanity check: try made up review

In [9]:
# load word2vec model
model = KeyedVectors.load('word2vec/word2vec-google-news-300.model') # Load word2vec model

In [19]:
madeup = ['i liked their pizza']
lstm_model(process_reviews_w2v(madeup, model))

Fetching review embeddings: 100%|██████████| 1/1 [00:00<?, ?it/s]


tensor([[0.2764, 0.0091, 0.7049, 0.0097]], grad_fn=<SoftmaxBackward0>)

The model is not able to correctly classify any common sense test cases, except for text that are both food and service. Therefore, we now focus on finetuning a BERT model instead of training an LSTM from scratch.